In [3]:
seed = 1
import numpy as np

def seed_torch(seed=1029):
    np.random.seed(seed)
    torch.manual_seed(seed)



In [1]:
import torch.nn as nn
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # One time step
        # We need to detach the hidden state to prevent exploding/vanishing gradients
        # This is part of truncated backpropagation through time (BPTT)
        out, hn = self.rnn(x, h0.detach())

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out) 
        # out.size() --> 100, 10
        return out

In [4]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tqdm import tqdm
import torch
import xlrd
import pandas as pd

book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(1,6)] for r in range(2,74)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)


book = xlrd.open_workbook('generateddatakfold.xlsx')
sheet = book.sheet_by_name('Sheet1')
replicated_data = [[sheet.cell_value(r, c)
         for c in range(0,5)] for r in range(0,7800)]

def create_inout_sequences(input_data, tw):
    training_seq = []
    label_seq = []
    L = len(input_data)
    for i in range(0,L,13):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+1:i+tw+1,0:3]
        training_seq.append(train_seq)
        label_seq.append(train_label)
    return training_seq,label_seq

standardised_data = scaler.transform(replicated_data)
array1 = standardised_data[:, 0:5]
array2 = standardised_data[:, 5:8]
x = torch.tensor(array1)

z, t = create_inout_sequences(x, 12)
x_train = torch.stack(z)
y_train = torch.stack(t)

n_splits = 6
splits = list(KFold(n_splits=n_splits, shuffle=False, random_state=seed)
              .split(x_train, y_train))
print(splits)



[(array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
       191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203,
       204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
       217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,
       230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242,
       243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255,
       256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268,
       269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 

       591, 592, 593, 594, 595, 596, 597, 598, 599]))]


In [8]:
import copy
i = 5
test_batchsizes = [1,2,3,4,5,6,7,8,9,10,20,40,60,100]
train_epochs = 20
learning_rate = 0.001
input_dim = 5
hidden_dim = 20
layer_dim = 2  
output_dim = 3
model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
init_state_opt = copy.deepcopy(optimizer.state_dict())
for batchsize in test_batchsizes:
            input_dim = 5
            hidden_dim = 20
            layer_dim = 2  
            output_dim = 3
            model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
            loss_fn = torch.nn.MSELoss(reduction='mean')
            total_loss = 0
            init_state = copy.deepcopy(model.state_dict())
            for i, (train_idx, valid_idx) in enumerate(splits):
                model.load_state_dict(init_state)
                optimizer.load_state_dict(init_state_opt)
                x_train_fold = torch.tensor(x_train[train_idx], dtype=torch.float32)
                y_train_fold = torch.tensor(y_train[train_idx], dtype=torch.float32)
                x_val_fold = torch.tensor(x_train[valid_idx], dtype=torch.float32)
                y_val_fold = torch.tensor(y_train[valid_idx], dtype=torch.float32)
                train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
                valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batchsize,
                                                           shuffle=True)
                valid_loader = torch.utils.data.DataLoader(valid, batch_size=batchsize,
                                                           shuffle=False)
                for epoch in range(train_epochs):
                    model.train()
                    avg_loss = 0.
                    for x_batch, y_batch in train_loader:
                        y_pred = model(x_batch)
                        loss = loss_fn(y_pred, y_batch)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                model.eval()
                avg_val_loss = 0.
                for x_batch, y_batch in valid_loader:
                    y_pred = model(x_batch).detach()
                    avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader.dataset)
                    final_loss = avg_val_loss
                total_loss += final_loss/(n_splits)
            print(batchsize)
            print('total loss={:.4f}'.format(total_loss))


C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gabriel\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().det

1
total loss=1.3344
2
total loss=1.2722
3
total loss=1.2422
4
total loss=1.3043
5
total loss=1.0943
6
total loss=1.3186
7
total loss=1.0703
8
total loss=0.9904
9
total loss=1.1714
10
total loss=1.2046
20
total loss=1.1994
40
total loss=1.3795
60
total loss=1.3849
100
total loss=1.0398


In [7]:
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/ANN3.pt"))                
                                